In [1]:
import pandas as pd
import os

def read_all_csv_from_directory(directory):
    csv_files = [f for f in os.listdir(directory) if f.endswith('.csv')]
    # errors 인자 제거, 여러 인코딩 시도
    df_list = []
    for f in csv_files:
        file_path = os.path.join(directory, f)
        try:
            df = pd.read_csv(file_path, encoding='utf-8')
        except UnicodeDecodeError:
            try:
                df = pd.read_csv(file_path, encoding='euc-kr')
            except UnicodeDecodeError:
                df = pd.read_csv(file_path, encoding='cp949')
        df_list.append(df)
    if df_list:
        return pd.concat(df_list, ignore_index=True)
    else:
        return pd.DataFrame()

In [2]:
# Example usage:
# df = read_all_csv_from_directory('path/to/your/csv_directory')

In [3]:
def read_all_csv_from_directory(directory):
    csv_files = [f for f in os.listdir(directory) if f.endswith('.csv')]
    # errors 인자 제거, 여러 인코딩 시도
    df_list = []
    for f in csv_files:
        file_path = os.path.join(directory, f)
        try:
            df = pd.read_csv(file_path, encoding='utf-8')
        except UnicodeDecodeError:
            try:
                df = pd.read_csv(file_path, encoding='euc-kr')
            except UnicodeDecodeError:
                df = pd.read_csv(file_path, encoding='cp949')
        df_list.append(df)
    if df_list:
        return pd.concat(df_list, ignore_index=True)
    else:
        return pd.DataFrame()

In [4]:
def read_excel_file(filepath):
    """
    Reads an Excel file from the given filepath and returns a pandas DataFrame.
    """
    return pd.read_excel(filepath)

In [5]:
def add_location_and_station_info(df, latitude_col='latitude', longitude_col='longitude', station_name_col='대여소', station_num_col='대여구 번호'):
    """
    Adds a 'location' column by combining latitude and longitude,
    and adds 'station_name' and 'station_number' columns.
    """
    df['location'] = df[latitude_col].astype(str) + ', ' + df[longitude_col].astype(str)
    df['station_name'] = df[station_name_col]
    df['station_number'] = df[station_num_col]
    return df

## 함수형 처리 예시
위에서 정의한 함수들을 활용하여 데이터프레임을 처리하는 예시입니다.

In [6]:
# 엑셀 파일 읽기
df_excel = read_excel_file(r'bike_rent_location.xlsx')

# CSV 디렉토리에서 데이터프레임 생성
df_combined = read_all_csv_from_directory('./')

# df_excel에 위치 정보 및 대여소 정보 추가 (필요시)
df_excel = add_location_and_station_info(
    df_excel,
    latitude_col='위도',
    longitude_col='경도',
    station_name_col='대여소명',
    station_num_col='대여소번호'  # 컬럼명 일치: '대여소 번호' → '대여소번호'
)

# 대여소 번호 기준 위도, 경도 추출
station_coords = df_excel[['대여소번호', '위도', '경도']].drop_duplicates()

# df_combined 컬럼명 확인
print(df_combined.columns)

# '대여 대여소 번호' 컬럼이 없으면 '대여 대여소번호'로부터 생성
if '대여 대여소 번호' not in df_combined.columns:
    if '대여 대여소번호' in df_combined.columns:
        df_combined = df_combined.rename(columns={'대여 대여소번호': '대여 대여소 번호'})
    else:
        raise KeyError("대여 대여소 번호 컬럼이 존재하지 않습니다.")

# '반납 대여소 번호' 컬럼이 없으면 생성 (예: 결측치 또는 대여 대여소 번호와 동일하게 복사)
if '반납 대여소 번호' not in df_combined.columns:
    if '반납 대여소번호' in df_combined.columns:
        df_combined = df_combined.rename(columns={'반납 대여소번호': '반납 대여소 번호'})
    else:
        df_combined['반납 대여소 번호'] = df_combined['대여 대여소 번호']

# 컬럼 타입 일치
df_combined['대여 대여소 번호'] = pd.to_numeric(df_combined['대여 대여소 번호'], errors='coerce').astype('Int64')
df_combined['반납 대여소 번호'] = pd.to_numeric(df_combined['반납 대여소 번호'], errors='coerce').astype('Int64')

# df_combined에 대여, 반납 대여소 번호 기준으로 위도, 경도 추가
df_combined = df_combined.merge(
    station_coords.rename(columns={'대여소번호': '대여 대여소 번호', '위도': '대여 위도', '경도': '대여 경도'}),
    on='대여 대여소 번호',
    how='left'
)
df_combined = df_combined.merge(
    station_coords.rename(columns={'대여소번호': '반납 대여소 번호', '위도': '반납 위도', '경도': '반납 경도'}),
    on='반납 대여소 번호',
    how='left'
)

Index(['자전거번호', '대여일시', '대여 대여소번호', '대여 대여소명', '이용시간', '이용거리'], dtype='object')


In [7]:
# 데이터프레임 전체 출력 설정
import pandas as pd
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

# df_excel 전체 출력
print("==== df_excel 전체 출력 ====")
print(df_excel)

==== df_excel 전체 출력 ====
        구분  대여소번호                                             대여소명         위도  \
0      마포구    101                                 101. (구)합정동 주민센터  37.549561   
1      마포구    102                                  102. 망원역 1번출구 앞  37.556000   
2      마포구    103                                  103. 망원역 2번출구 앞  37.554951   
3      마포구    104                                  104. 합정역 1번출구 앞  37.550629   
4      마포구    105                                  105. 합정역 5번출구 앞  37.550007   
5      마포구    106                                  106. 합정역 7번출구 앞  37.548645   
6      마포구    107                             107. 신한은행 서교동금융센터점 앞  37.557510   
7      마포구    108                                     108. 서교동 사거리  37.552746   
8      마포구    109                                      109. 제일빌딩 앞  37.547691   
9     서대문구    110                                         110. 사천교  37.568199   
10     마포구    111                                  111. 상수역 2번출구 앞  37.547871   
11 

In [8]:
# station_coords 전체 출력
print("==== station_coords 전체 출력 ====")
print(station_coords)

==== station_coords 전체 출력 ====
      대여소번호         위도          경도
0       101  37.549561  126.905754
1       102  37.556000  126.910454
2       103  37.554951  126.910835
3       104  37.550629  126.914986
4       105  37.550007  126.914825
5       106  37.548645  126.912827
6       107  37.557510  126.918503
7       108  37.552746  126.918617
8       109  37.547691  126.919983
9       110  37.568199  126.917847
10      111  37.547871  126.923531
11      112  37.549202  126.923203
12      113  37.557499  126.923805
13      114  37.557060  126.924423
14      115  37.558933  126.927116
15      116  37.564541  126.927071
16      117  37.591160  126.941330
17      118  37.547733  126.931763
18      119  37.545284  126.931053
19      120  37.545242  126.934113
20      121  37.549767  126.933174
21      122  37.547249  126.939293
22      123  37.594330  126.947388
23      124  37.551140  126.936989
24      125  37.549484  126.938950
25      126  37.550411  126.943848
26      127  37.553520  

In [9]:
# df_combined 전체 출력 (행이 많으므로 head/tail만 출력 권장)
print("==== df_combined 일부 출력 (상위 100행) ====")
print(df_combined.head(100))

print("==== df_combined 일부 출력 (하위 100행) ====")
print(df_combined.tail(100))

print("==== df_combined shape ====")
print(df_combined.shape)

==== df_combined 일부 출력 (상위 100행) ====
        자전거번호              대여일시  대여 대여소 번호            대여 대여소명  이용시간   이용거리  \
0   SPB-22040   2019-06-03 8:49        646  장한평역 1번출구 (국민은행앞)    27   1330   
1   SPB-07446   2019-06-03 8:33        526           용답토속공원 앞    54   1180   
2   SPB-20387   2019-06-05 8:27        646  장한평역 1번출구 (국민은행앞)    12   1930   
3   SPB-16794   2019-06-05 8:46        646  장한평역 1번출구 (국민은행앞)     6   1340   
4   SPB-18266   2019-06-10 8:27        529       장한평역 8번 출구 앞     5   1230   
5   SPB-13926   2019-06-11 8:29        646  장한평역 1번출구 (국민은행앞)     7   1360   
6   SPB-14638   2019-06-12 8:29        646  장한평역 1번출구 (국민은행앞)     5   1340   
7   SPB-18588   2019-06-17 8:34        646  장한평역 1번출구 (국민은행앞)     8   1360   
8   SPB-21148   2019-06-17 8:47        646  장한평역 1번출구 (국민은행앞)    22   1330   
9   SPB-24533   2019-06-18 8:36        646  장한평역 1번출구 (국민은행앞)     6   1230   
10  SPB-24350   2019-06-18 8:46        646  장한평역 1번출구 (국민은행앞)    11   1200   
11  SPB-22669   2019-06-19